# Final code

In [ ]:
!pip install langchain_google_genai langchain_ollama python-dotenv

In [2]:
# імпорти
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from typing_extensions import Literal
import pandas as pd
import time
import os
from langchain_ollama.llms import OllamaLLM
from dotenv import load_dotenv

In [3]:
# Завантажуємо .env файл
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")

In [ ]:
# Налаштування моделі
model_name = "gemini-2.0-flash-lite"
llm = ChatGoogleGenerativeAI(
    model=model_name,
    api_key=api_key,
    temperature=0.8,
    max_retries=2,
    top_p=0.9,
    top_k=40,
)

llm = OllamaLLM(
    base_url=os.environ["BASE_URL"],
    model="aya-expanse:32b-q2_K",
    keep_alive="3h",
    temperature=0.7,
    format="json",
    top_p=0.9
)

In [39]:
# Ініціалізація моделі
class OutputModel(BaseModel):
    title: str = Field(description="Заголовок новини")
    text: str = Field(description="Текст новини (300-500 слів)")
    topic: str = Field(description="Обрана тема новини")
    type_news: str = Field(description="Обраний тип новини")

parser = JsonOutputParser(pydantic_object=OutputModel)

In [43]:
# System message
system_message = """Ви — досвідчений журналіст із понад 10 роками роботи в провідних українських та міжнародних медіа.
Ваші тексти вирізняються глибиною аналізу, структурованістю та дотриманням високих стандартів журналістики.
Ви володієте майстерністю фактчекінгу, балансу думок і доступного викладу складних тем.\
"""

# Human message
human_message_template = """Згенеруй **фейкову, але правдоподібну** новину у стилі провідних українських медіа 2022–2025 років. \
Тематика — **Політика**, **Економіка**, **Суспільство** або **Військові події**.

---

### **Контекст подачі:**
- Тональність: {tone}
- Стиль: {style}


---

### **Тип новини: **
Новина мати тип: {type_of_news}

---

### Вимоги до тексту:
- **Почни з головного** — подія, її суть, причетні особи або інституції.
- Пиши **доступною, живою мовою** — без офіціозу, штампів чи технократичного жаргону.
- Стиль має бути **максимально реалістичним**, у форматі репортажу або аналітичної замітки.
- Включай **реальні імена посадовців, державних установ і фактів** (у межах 2022–2025 рр.).
- Додай **коментарі експертів або посадових осіб** — вигадані, але переконливі.
- **Не вигадуй географічних назв**, не використовуй очевидні кліше чи фантастичні події.
- Орієнтуйся на стиль **«Української правди», «bihus.info», «ТСН»** — стриманий, динамічний, фактологічний.

---

# ВИВІД МАЄ БУТИ У ФОРМАТІ:
{format_of_output}
"""

In [50]:
import random
from langchain_core.prompts import ChatPromptTemplate

tones = ["нейтральна", "тривожна", "заспокійлива"]
styles = ["суха подача", "драматична", "аналітична", "іронічна"]
type_of_news= ['Маніпуляція', 'Дезінформація', 'Пропаганда']

# Генерація новини
def generate_one_news():
    prompt = ChatPromptTemplate(
        [
            ("system", system_message),
            ("human", human_message_template)
        ]
    )

    chain = prompt | llm | parser

    # Рандомно вибираємо тон, стиль, тип новин
    tone = random.choice(tones)
    style = random.choice(styles)
    type = random.choice(type_of_news)

    input_data = {
        "tone": tone,
        "style": style,
        "type_of_news": type,
        "format_of_output": parser.get_format_instructions()
    }

    # Запускаємо модель
    return chain.invoke(input_data)['properties']

In [ ]:
result = generate_one_news()['properties']
result

{'properties': {'title': 'Скандал у Верховній Раді: депутат звинувачує уряд у корупції',
  'text': "У Верховній Раді сьогодні розгорівся гучний скандал. Депутат від опозиційної фракції 'Соціально-демократичний союз' Іван Петров звинуватив уряд у систематичній корупції та зловживанні владою. Петрова підтримали кілька інших депутатів, які стверджують, що мають докази неправомірних дій високопосадовців.\n\nПід час емоційного виступу на пленарному засіданні Петров заявив, що прем'єр-міністр Ольга Васильєва та кілька її заступників причетні до схеми з розкрадання державних коштів, призначених для відновлення інфраструктури в постраждалих від війни регіонах. За словами депутата, сума збитків становить мільярди гривень.\n\n'Я маю документи, які підтверджують факт незаконного збагачення та зловживання владою на найвищому рівні', - заявив Петров, демонструючи копії документів, що нібито викривають корупційну схему. Він пообіцяв передати ці матеріали до правоохоронних органів та вимагати негайно

In [51]:
# додавання до датасету
def append_to_csv(news, path= "..\\..\\data\\fake_news_dataset.csv"):
    df = pd.DataFrame(
        [
            {
                "title": news['title'],
                "text": news['text'],
                "topic": news['topic'],
                "type": news['type_news']
            }
        ]
    )

    if not os.path.exists(path):
        df.to_csv(path, index=False, encoding="utf-8")
    else:
        df.to_csv(path, mode="a", index=False, header=False, encoding="utf-8")

In [52]:
n = 5
delay = 2

for i in range(n):
    news = generate_one_news()
    print(f"[{i+1}/{n}] {news['title']}")
    append_to_csv(news)
    # time.sleep(delay)

print(f"\n Згенеровано {n} новин. Результати збережено.")

[1/5] Міністр оборони підозрюється у корупції: хто замінить генерала Тарана?
[2/5] Сенсаційна заява міністра оборони: 'Україна готова до можливого вторгнення з боку РФ'
[3/5] Дезінформація про виведення військ РФ з Херсона спричинила паніку серед населення
[4/5] Дезінформація: ЗМІ розкривають фейкову операцію з дискредитації українського уряду
[5/5] Україна посилює оборону: нова військова стратегія

 Згенеровано 5 новин. Результати збережено.


In [53]:
import pandas as pd

df = pd.read_csv("..\\..\\data\\fake_news_dataset.csv")
df

,title,text,topic,type
0,ЗМІ: Європейські партнери висловлюють стурбова...,"Київ, Україна – Останніми тижнями зростає зане...",Політика,Маніпуляція
1,ЗМІ: Відставка Резнікова стала результатом нез...,"Київ, Україна – За інформацією, отриманою з кі...",Політика,Маніпуляція
2,Злив даних з Офісу Президента: чия вина та які...,Несподіваний витік внутрішньої документації з ...,Політика,Маніпуляція
3,Зростання експорту зерна через Польщу: чи не х...,Київ – Останні місяці демонструють стрімке зро...,Економіка,Маніпуляція
4,Зростання імпорту російського газу через посер...,"Київ, Україна – Останні місяці спостерігається...",Економіка,Маніпуляція
5,Міністр оборони підозрюється у корупції: хто з...,"У середу, 2023 року, українські ЗМІ вибухнули ...",Політика,Маніпуляція
6,Сенсаційна заява міністра оборони: 'Україна го...,"Міністр оборони України, Олексій Громов, зроби...",Політика,Дезінформація
7,Дезінформація про виведення військ РФ з Херсон...,У соціальних мережах та месенджерах поширилася...,Політика,Дезінформація
8,Дезінформація: ЗМІ розкривають фейкову операці...,"У п'ятницю, 20 березня 2025 року, кілька прові...",Політика,Дезінформація
9,Україна посилює оборону: нова військова стратегія,Уряд України оголосив про амбітний план модерн...,"Політика, Військові події",Пропаганда


In [54]:
print(df['text'][0])

Київ, Україна – Останніми тижнями зростає занепокоєння серед західних партнерів України щодо прогресу в проведенні ключових реформ, зокрема у сфері боротьби з корупцією та прозорості державних закупівель. Джерела в дипломатичних колах ЄС та США повідомляють про низку неформальних зустрічей, на яких обговорювалася ситуація з виконанням Україною рекомендацій Європейської комісії щодо членства в ЄС.

Особливу увагу привертає затримка з призначенням нового керівника Національного антикорупційного бюро України (НАБУ) та невирішеність питання щодо реформування судової системи. "Ми бачимо позитивні зрушення, але темп реформ не відповідає критично важливим викликам, з якими Україна стикається, зокрема, напередодні зими", - цитує джерело в Брюсселі анонімного представника Європейської комісії.

Ситуацію ускладнює також питання прозорості використання фінансової допомоги, що надходить від міжнародних партнерів. Незважаючи на офіційні заяви української влади про ефективний контроль за розподілом 